Environment Infomation:
* Win10 x64
* Python 3.8.8
* model dependencies:
    * `numpy==1.19.2`
    * `pandas==1.3.2`
    * `fastparquet==0.7.1`
    * `joblib==1.0.1`
    * `scikit-learn==0.24.2`
    * `tensorflow==2.5.0`
    * `keras==2.4.3`
* API dependencies:
    * `pydantic==1.8.2`
    * `fastapi==0.68.1`
    * `uvicorn==0.15.0`
* Query dependencies:
    * `requests==2.26.0`
* Unit Test dependencies:
    * `nose==1.3.7`

## Model Servering (Windows Comand Line)

### Start the Service
**Using uvicorn**
* Activate the python environment with desired dependencies
* Navigate to the `root/` directory, which contains `main_recommender.py`
* Execute `uvicorn main_recommender:app --reload --host 127.0.0.1 --port 8000`

**Using docker image**
* Navigate to the `root/image/"`directory, which contains the image `main_recommender.tar`
* Load the image by executing `docker load -i main_recommender.tar`
* Spin up the container by executing `docker run -d -p 8000:8000 --name yangxi-recommender main_recommender`

Once started, the service can be reached at `http://127.0.0.1:8000`

### Service Endpoints
There are two endpoints:
* `/predict`
    * POST with a single key-value pair. For example: `{"user_id":229957}`
        * key: `"user_id"`
        * value: a integer representing the user's ID to query the recommendations
    * Response four key-value pairs.<br>
    For example: `{'user_type':'existing', 'merchant_id':'90', 'ar_merchant_id':'90|326|264|144|272', 'ar_rating':'4.572|4.540|4.449|nan|nan'}`
        * `"user_type"`: `"existing"` or `"new"` indicating whether the recommender treat this users as an existing customer or a new customer.
        * `"merchat_id"`: a string representing the top recommended merchant ID
        * `"ar_merchat_id"`: a pipe-delimited string representing all 5 merchant IDs recommended to the user
        * `"ar_rating"`: a pipe-delimited string representing the predicted rating of the recommendations, sorted from high to low.<br>
        `nan` value means that the recommendation is not generated by the machine learning model.

<br>

* `/batch_predict`
    * POST with a single key-value pair. For example: `{"list_user_id":[229957, 206475]}`
        * key: `"list_user_id"`
        * value: a list of integers representing the user IDs to query the recommendations
    * Response four key-value pairs.<br>
    For example: `{'user_type':'existing,new', 'merchant_id':'90,398', 'ar_merchant_id':'90|326|264|144|36,398|490|271|472|147', 'ar_rating':'4.572|4.540|4.449|nan|nan,nan|nan|nan|nan|nan'}`
        * `"user_type"`: a comma-delimited string with the `user_type` of each `user_id` queried.
        * `"merchat_id"`: a comma-delimited string with the `merchat_id` of each `user_id` queried.
        * `"ar_merchat_id"`: a comma-delimited string with the `ar_merchat_id` of each `user_id` queried.
        * `"ar_rating"`: a comma-delimited string with the `ar_rating` of each `user_id` queried.

### Stop the Service
* If started by uvicorn: `Ctrl+C` to stop the process
* If started by docker: `docker stop yangxi-recommender`

To execute the following sections, the API needs to be startup first.

# Sample Query

In [1]:
import requests

response = requests.get('http://127.0.0.1:8000')
response.json()

{'message': 'Merchant Recommender'}

### Endpoint "predict": for a single user ID

In [2]:
response = requests.post('http://127.0.0.1:8000/predict', data='{"user_id":229957}')
response.json()

{'user_type': 'existing',
 'merchant_id': '90',
 'ar_merchant_id': '90|326|264|144|147',
 'ar_rating': '4.572|4.540|4.449|nan|nan'}

In [3]:
response = requests.post('http://127.0.0.1:8000/predict', data='{"user_id":206475}')
response.json()

{'user_type': 'new',
 'merchant_id': '149',
 'ar_merchant_id': '149|490|397|365|36',
 'ar_rating': 'nan|nan|nan|nan|nan'}

### Endpoint "batch_predict": for a list of user IDs

In [4]:
response = requests.post('http://127.0.0.1:8000/batch_predict', data='{"list_user_id":[229957, 206475]}')
response.json()

{'user_type': 'existing,new',
 'merchant_id': '90,490',
 'ar_merchant_id': '90|326|264|149|147,490|365|398|472|261',
 'ar_rating': '4.572|4.540|4.449|nan|nan,nan|nan|nan|nan|nan'}

# Unit Test

In [5]:
import requests
from nose.tools import assert_equal

# Single prediction at /predict endpoint
def UnitTestPredict(url='http://127.0.0.1:8000', data='{"user_id":229957}'):
    print('Testing single predict at /predict endpoint...')
    response = requests.post(url+'/predict', data=data)
    dResponse = response.json()

    try:
        assert_equal(list(dResponse.keys()), ['user_type', 'merchant_id', 'ar_merchant_id', 'ar_rating'])
    except:
        raise Exception('predict: Mismatched response keys')

    try:
        assert_equal(dResponse['user_type'] in ['existing','new'], True)
    except:
        raise Exception('predict: user_type should be one of "existing", "new"')

    try:
        assert_equal(type(int(dResponse['merchant_id'])), int)
    except:
        raise Exception('predict: merchant_id should be a string representation of a single integer')

    try:
        assert_equal([type(int(x)) for x in dResponse['ar_merchant_id'].split('|')], [int]*5)
    except:
        raise Exception('predict: ar_merchant_id should be a string reprenting 5 integers seperated by "|"')

    try:
        assert_equal([type(float(x)) for x in dResponse['ar_rating'].split('|')], [float]*5)
    except:
        raise Exception('predict: ar_rating should be a string reprenting 5 floating point numbers seperated by "|"')

    print('ALL TESTS PASSED.')


# Batch prediction at /batch_predict endpoint
def UnitTestBatchPredict(url='http://127.0.0.1:8000', data='{"list_user_id":[229957, 206475]}', nUsers=2):
    print('Testing batch predict at /batch_predict endpoint...')
    response = requests.post(url+'/batch_predict', data=data)
    dResponse = response.json()

    try:
        assert_equal(list(dResponse.keys()), ['user_type', 'merchant_id', 'ar_merchant_id', 'ar_rating'])
    except:
        raise Exception('batch_predict: Mismatched response keys')

    try:
        lUserType = dResponse['user_type'].split(',')
        assert_equal(len(lUserType), nUsers)
        assert_equal(sum([x in ['existing', 'new'] for x in lUserType]), nUsers)
    except:
        s = '''
        batch_predict:
        user_type should be a comma-delimited string representing a list with the same length as users queried.
        Each element should be one of "existing", "new".
        '''
        raise Exception(s)    

    try:
        lMerchantId = dResponse['merchant_id'].split(',')
        assert_equal(len(lMerchantId), nUsers)
        assert_equal([type(int(x)) for x in lMerchantId], [int]*nUsers)
    except:
        s = '''
        batch_predict:
        merchant_id should be a comma-delimited string representing a list with the same length as users queried.
        Each element should be a string representation of a single integer.
        '''
        raise Exception(s)    


    try:
        lArMerchantId = dResponse['ar_merchant_id'].split(',')
        assert_equal(len(lArMerchantId), nUsers)
        assert_equal([[type(int(x)) for x in s.split('|')] for s in lArMerchantId], [[int]*5]*nUsers)
    except:
        s = '''
        batch_predict:
        ar_merchant_id should be a comma-delimited string representing a list with the same length as users queried.
        Each element should be a string reprenting 5 integers seperated by "|".
        '''
        raise Exception(s)

    try:
        lArRating = dResponse['ar_rating'].split(',')
        assert_equal(len(lArRating), nUsers)
        assert_equal([[type(float(x)) for x in s.split('|')] for s in lArRating], [[float]*5]*nUsers)
    except:
        s = '''
        batch_predict:
        ar_rating should be a comma-delimited string representing a list with the same length as users queried.
        Each element should be a string reprenting 5 floating point numbers seperated by "|".
        '''
        raise Exception(s)
    
    print('ALL TESTS PASSED.')


In [6]:
UnitTestPredict()

Testing single predict at /predict endpoint...
ALL TESTS PASSED.


In [7]:
UnitTestBatchPredict()

Testing batch predict at /batch_predict endpoint...
ALL TESTS PASSED.
